In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
    "X": "something"
}
step_params={
    "X": "something_else"
}
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    
    outputs = 
    [
        { ENTITY_NAME: "bento_service" }
    ],

)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
# Train
import pytorch_lightning as pl
import numpy as np
import torch
from torch.nn import MSELoss
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

from bentoservice import SimplePytorchLightning


class SimpleDataset(Dataset):
    def __init__(self):
        X = np.arange(10000)
        y = X * 2
        X = [[_] for _ in X]
        y = [[_] for _ in y]
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return {"X": self.X[idx], "y": self.y[idx]}


class MyModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(1, 1)
        self.criterion = MSELoss()

    def forward(self, inputs_id, labels=None):
        outputs = self.fc(inputs_id)
        loss = 0
        if labels is not None:
            loss = self.criterion(outputs, labels)
        return loss, outputs

    def train_dataloader(self):
        dataset = SimpleDataset()
        return DataLoader(dataset, batch_size=1000)

    def training_step(self, batch, batch_idx):
        input_ids = batch["X"]
        labels = batch["y"]
        loss, outputs = self(input_ids, labels)
        return {"loss": loss}

    def configure_optimizers(self):
        optimizer = Adam(self.parameters())
        return optimizer

model = MyModel()
trainer = pl.Trainer(max_epochs=20, gpus=1)
trainer.fit(model)

# Create a iris classifier service with the newly trained model
simple_pytorch_lightning_service = SimplePytorchLightning()
simple_pytorch_lightning_service.pack("model", model)

In [ ]:
# Test model API
from sinara.bentoml import start_dev_bentoservice, stop_dev_bentoservice

# Test invoking BentoService instance
X = torch.Tensor([[1.0], [51.0], [89.0]])
_, y = model(X)
print(y)

# Start a dev model server to test out the API endpoint locally
start_dev_bentoservice(simple_pytorch_lightning_service)

# Stop the dev model server
stop_dev_bentoservice(simple_pytorch_lightning_service)

In [ ]:
# Serve model
from sinara.bentoml import save_bentoservice

outputs = substep.outputs()

# Save the entire prediction service to a BentoML bundle
save_bentoservice(simple_pytorch_lightning_service, path=outputs.bento_service, service_version=f"{outputs.fullname('bento_service')}.{substep.run_id}")
 